# RNA-seq Quantification and Differential Gene Expression

This notebook encapsulates the quantification of RNA-seq experiments using [Salmon](https://combine-lab.github.io/salmon/) (Patro et al.), the computation of differentially expressed genes using [DESeq2](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8) (Love et al.), and identification of enriched gene sets with [GSEA](https://gsea-msigdb.org/) (Subramanian etl al.) or ssGSEA (Barbie et al.). These tools are implemented as GenePattern modules, enabling the use of a modern end-to-end gene expression analysis pipeline without local installations or compute resources.

## Log in to GenePattern

You must log in to a GenePattern server. In this notebook we will use **```GenePattern Beta``` **

Note: if you are not familiar with GenePattern Notebook features, you can review them here: <a href="https://notebook.genepattern.org/services/sharing/notebooks/361/preview/">GenePattern Notebook Tutorial</a>.
<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Sign in to GenePattern by entering your username and password into the form below.
</div>

In [9]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://beta.genepattern.org/gp", "", ""))

GPAuthWidget()

# Indexing and Pseudoalignment using Salmon

## Index a transcriptome with salmon.index

The first step of the Salmon pipeline is to create sample-independent index of the transcriptiome, as well as "decoy" sequences from the genome. Building a decoy-aware index allows Salmon to mitigate potential spurious mapping of reads that actually arise from some unannotated genomic locus that is sequence-similar to an annotated transcriptome.

The inpact of decoy-aware indexing is described in the following pre-print from the Salmon authors: [Alignment and mapping methodology influence transcript abundance estimation](https://www.biorxiv.org/content/10.1101/657874v1).

An index, once constructed, can be reused for future experiments utilizing the same organism.

In [14]:
salmon_index_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:999179')
salmon_index_job_spec = salmon_index_task.make_job_spec()
salmon_index_job_spec.set_parameter("GTF.gz", "")
salmon_index_job_spec.set_parameter("Transcriptome.fa.gz", "")
salmon_index_job_spec.set_parameter("Genome.fa.gz", "")
salmon_index_job_spec.set_parameter("kmer", "31")
salmon_index_job_spec.set_parameter("Index.Mode", "full")
salmon_index_job_spec.set_parameter("output.index.name", "<GTF.gz_basename>")
salmon_index_job_spec.set_parameter("job.memory", "2 Gb")
salmon_index_job_spec.set_parameter("job.queue", "gp-new-beta")
salmon_index_job_spec.set_parameter("job.cpuCount", "1")
salmon_index_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(salmon_index_task)


GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:9991…

## Quantify against the index with salmon.quant

Salmon will compute the mappings of raw reads against the transcriptome+decoy index, and produce estimations of the abundance of each transcript for each sample. The output is a single "_.quant.sf_" file for each sample, as well as an associated gzipped directory contaning detailed information from the alignment, and, if selected, bootstrapping estimations. If bootstrapping or gibbs sampling is run this gzipped directory contains the information needed to run [sleuth](https://pachterlab.github.io/sleuth/about) using the [wasabi](https://github.com/COMBINE-lab/wasabi) importer.

In [15]:
salmon_quant_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:2991')
salmon_quant_job_spec = salmon_quant_task.make_job_spec()
salmon_quant_job_spec.set_parameter("Reads", "")
salmon_quant_job_spec.set_parameter("Transcriptome.Index", "")
salmon_quant_job_spec.set_parameter("Library.Type", "A")
salmon_quant_job_spec.set_parameter("Sampling", "BOOT")
salmon_quant_job_spec.set_parameter("seqBias", "true")
salmon_quant_job_spec.set_parameter("gcBias", "true")
salmon_quant_job_spec.set_parameter("posBias", "true")
salmon_quant_job_spec.set_parameter("N.Sampling", "30")
salmon_quant_job_spec.set_parameter("Mimic.Bowtie", "off")
salmon_quant_job_spec.set_parameter("Recover.Orphans", "false")
salmon_quant_job_spec.set_parameter("Hard.Filter", "false")
salmon_quant_job_spec.set_parameter("Allow.Dovetail", "false")
salmon_quant_job_spec.set_parameter("Dump.EQ", "false")
salmon_quant_job_spec.set_parameter("reduce.GC.Memory", "false")
salmon_quant_job_spec.set_parameter("Range.Factorization", "false")
salmon_quant_job_spec.set_parameter("Range.Factorization.Bins", "4")
salmon_quant_job_spec.set_parameter("biasSpeedSamp", "false")
salmon_quant_job_spec.set_parameter("biasSpeedSamp.Factor", "5")
salmon_quant_job_spec.set_parameter("job.memory", "2 Gb")
salmon_quant_job_spec.set_parameter("job.queue", "gp-new-beta")
salmon_quant_job_spec.set_parameter("job.cpuCount", "1")
salmon_quant_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(salmon_quant_task)


GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:2991…

# Collapse transcript-level quantifications to gene-level with tximport, and compute differential gene expression with DESeq2

Using tximport, transcript-level quantifications from Salmon can be extracted and collapsed to gene level, allowing for differential gene expression analayis using classical tools like DESeq2 (Love et al.). 

The tximport.DESeq2.Normalize module takes transcript-level quantifications from salmon, and other tools, and outputs data at gene-level. 

Output options are "normalized counts" which are counts normalized for between-sample comparisons utilizing the DESeq2 algorithm (suitable for GSEA), or within-sample normalized TPM values (suitable for ssGSEA). If a Sample Info file is provided, this module will also output DESeq2's differential expression calculations. Additionally, if the Sample Info file is provided, phenotype information will be used to produce a formatted CLS file for use with GSEA.

<b>Note: </b> Be sure to set "Quant Type" to "Salmon .quant.sf".

In [16]:
tximport_deseq2_normalize_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:179')
tximport_deseq2_normalize_job_spec = tximport_deseq2_normalize_task.make_job_spec()
tximport_deseq2_normalize_job_spec.set_parameter("Quantifications", "")
tximport_deseq2_normalize_job_spec.set_parameter("Sample.Info", "")
tximport_deseq2_normalize_job_spec.set_parameter("Quant.Type", "RSEM")
tximport_deseq2_normalize_job_spec.set_parameter("Transcriptome.Database", "")
tximport_deseq2_normalize_job_spec.set_parameter("Output.Normalized.Counts", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Output.TPM", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("output.file.base", "Merged.Data")
tximport_deseq2_normalize_job_spec.set_parameter("Reverse.Sign", "FALSE")
tximport_deseq2_normalize_job_spec.set_parameter("Annotate.DEGs", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("Split.Identifiers", "TRUE")
tximport_deseq2_normalize_job_spec.set_parameter("random.seed", "779948241")
tximport_deseq2_normalize_job_spec.set_parameter("job.memory", "2 Gb")
tximport_deseq2_normalize_job_spec.set_parameter("job.queue", "gp-new-beta")
tximport_deseq2_normalize_job_spec.set_parameter("job.cpuCount", "1")
tximport_deseq2_normalize_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(tximport_deseq2_normalize_task)


GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:179'…

# Run GSEA on Normalized Counts

Select the normalized counts output from the tximport.DESeq2 module in the drop-down for the expression dataset. If a sample info file was also provided, select the .cls file created for the phenotype labels drop-down, otherwise, upload a manually created phenotype labels cls.

Be sure to select the appropriate chip platform file for the transcriptome used in the Salmon quantification. For Gencode and Ensembl transcriptomes, this should be the species-sepecific _Ensembl_Gene_ID_ chip file. We also recomend setting the Advanced parameter "collapsing mode for probe sets with more than one match" to "sum_of_probes".

In [10]:
gsea_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00072')
gsea_job_spec = gsea_task.make_job_spec()
gsea_job_spec.set_parameter("expression.dataset", "")
gsea_job_spec.set_parameter("gene.sets.database", "")
gsea_job_spec.set_parameter("number.of.permutations", "1000")
gsea_job_spec.set_parameter("phenotype.labels", "")
gsea_job_spec.set_parameter("target.profile", "")
gsea_job_spec.set_parameter("collapse.dataset", "Collapse")
gsea_job_spec.set_parameter("permutation.type", "phenotype")
gsea_job_spec.set_parameter("chip.platform.file", "")
gsea_job_spec.set_parameter("scoring.scheme", "weighted")
gsea_job_spec.set_parameter("metric.for.ranking.genes", "Signal2Noise")
gsea_job_spec.set_parameter("gene.list.sorting.mode", "real")
gsea_job_spec.set_parameter("gene.list.ordering.mode", "descending")
gsea_job_spec.set_parameter("max.gene.set.size", "500")
gsea_job_spec.set_parameter("min.gene.set.size", "15")
gsea_job_spec.set_parameter("collapsing.mode.for.probe.sets.with.more.than.one.match", "Max_probe")
gsea_job_spec.set_parameter("normalization.mode", "meandiv")
gsea_job_spec.set_parameter("randomization.mode", "no_balance")
gsea_job_spec.set_parameter("omit.features.with.no.symbol.match", "true")
gsea_job_spec.set_parameter("make.detailed.gene.set.report", "true")
gsea_job_spec.set_parameter("median.for.class.metrics", "false")
gsea_job_spec.set_parameter("number.of.markers", "100")
gsea_job_spec.set_parameter("plot.graphs.for.the.top.sets.of.each.phenotype", "20")
gsea_job_spec.set_parameter("random.seed", "timestamp")
gsea_job_spec.set_parameter("save.random.ranked.lists", "false")
gsea_job_spec.set_parameter("create.svgs", "false")
gsea_job_spec.set_parameter("selected.gene.sets", "")
gsea_job_spec.set_parameter("output.file.name", "<expression.dataset_basename>.zip")
gsea_job_spec.set_parameter("alt.delim", "")
gsea_job_spec.set_parameter("create.zip", "true")
gsea_job_spec.set_parameter("create.gcts", "false")
gsea_job_spec.set_parameter("dev.mode", "false")
gsea_job_spec.set_parameter("job.memory", "8 Gb")
gsea_job_spec.set_parameter("job.queue", "gp-new-beta")
gsea_job_spec.set_parameter("job.cpuCount", "1")
gsea_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(gsea_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00072')

# Single sample GSEA using TPM normalized data

We will use the GenePattern ssGSEA analysis module to transform a gene expression dataset into a dataset where each row corresponds to a pathway, and each column is a sample. Each value in the new dataset will therefore represent the up- or downregulation of a pathway (row) within a sample (column).

## Collapse Dataset

In [11]:
gseacollapsedataset_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00357')
gseacollapsedataset_job_spec = gseacollapsedataset_task.make_job_spec()
gseacollapsedataset_job_spec.set_parameter("expression.dataset", "")
gseacollapsedataset_job_spec.set_parameter("chip.platform.file", "")
gseacollapsedataset_job_spec.set_parameter("collapsing.mode", "Max_probe")
gseacollapsedataset_job_spec.set_parameter("omit.features.with.no.symbol.match", "true")
gseacollapsedataset_job_spec.set_parameter("dev.mode", "false")
gseacollapsedataset_job_spec.set_parameter("job.memory", "2 Gb")
gseacollapsedataset_job_spec.set_parameter("job.queue", "gp-new-beta")
gseacollapsedataset_job_spec.set_parameter("job.cpuCount", "1")
gseacollapsedataset_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(gseacollapsedataset_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00357')

## Run ssGSEA

In [12]:
ssgsea_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')
ssgsea_job_spec = ssgsea_task.make_job_spec()
ssgsea_job_spec.set_parameter("input.gct.file", "")
ssgsea_job_spec.set_parameter("output.file.prefix", "")
ssgsea_job_spec.set_parameter("gene.sets.database.files", "")
ssgsea_job_spec.set_parameter("gene.symbol.column", "Name")
ssgsea_job_spec.set_parameter("gene.set.selection", "ALL")
ssgsea_job_spec.set_parameter("sample.normalization.method", "none")
ssgsea_job_spec.set_parameter("weighting.exponent", "0.75")
ssgsea_job_spec.set_parameter("min.gene.set.size", "10")
ssgsea_job_spec.set_parameter("combine.mode", "combine.add")
ssgsea_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_task)


GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00270')

# Differential ssGSEA with ssGSEA_ROC

We will use the GenePattern ssGSEA_ROC module, which implements the R package ROCR (Sing et al.) to analyze the coherence between an assigned phenotype for the samples (e.g. high or low expression of a specific marker gene) and enrichment of the selected gene sets. For each gene set, the Matthews correlation coefficient, the area under the curve (AUC), a two-sided wilcox test on the ssSGEA scores and a number of other metrics are calculated.

For datasets with ≥7 samples in both phenotypes, ssGSEA_ROC utilizes a modified version of the GSEA phenotype permutation algorithm to calcualte GSEA style NOM pValues and FDRs on the basis of permutation of the sample-to-phenotype assignments. 

In [13]:
ssgsea_roc_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177')
ssgsea_roc_job_spec = ssgsea_roc_task.make_job_spec()
ssgsea_roc_job_spec.set_parameter("PROJ.gct", "")
ssgsea_roc_job_spec.set_parameter("CLS", "")
ssgsea_roc_job_spec.set_parameter("Reverse", "FALSE")
ssgsea_roc_job_spec.set_parameter("Plot.Top.Results", "20")
ssgsea_roc_job_spec.set_parameter("job.memory", "2 Gb")
ssgsea_roc_job_spec.set_parameter("job.queue", "gp-new-beta")
ssgsea_roc_job_spec.set_parameter("job.cpuCount", "1")
ssgsea_roc_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(ssgsea_roc_task)


GPTaskWidget(lsid='urn:lsid:8080.gpserver.ip-172-31-26-71.ip-172-31-26-71.ec2.internal:genepatternmodules:177'…

# References

- Barbie DA, Tamayo P, et al. Systematic RNA interference reveals that oncogenic KRAS-driven cancers require TBK1. Nature. 2009;462:108-112. https://pubmed.ncbi.nlm.nih.gov/19847166/

- Chicco D, Jurman G. The advantages of the Matthews correlation coefficient (MCC) over F1 score and accuracy in binary classification evaluation. BMC Genomics 2020;21:6. https://doi.org/10.1186/s12864-019-6413-7

- Love MI, Huber W, Anders S. Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2. Genome Biol. 2014;15(12):550. https://doi.org/10.1186/s13059-014-0550-8

- Patro R, Duggal G, Love MI, Irizarry RA, Kingsford C. Salmon provides fast and bias-aware quantification of transcript expression. Nat Methods. 2017 Apr;14(4):417-419. Epub 2017 Mar 6. PMID: 28263959; https://doi.org/10.1038/nmeth.4197

- Sing T, Sander O, Beerenwinkel N, Lengauer T. ROCR: visualizing classifier performance in R. Bioinformatics. 2005;21(20):3940-3941. https://doi.org/10.1093/bioinformatics/bti623

- Subramanian A, Tamayo P, Mootha VK, Mukherjee S, Ebert BL, Gillette MA, Paulovich A, Pomeroy SL, Golub TR, Lander ES, Mesirov JP. Gene set enrichment analysis: A knowledge-based approach for interpreting genome-wide expression profiles. PNAS. 2005;102(43):15545-15550. http://www.pnas.org/content/102/43/15545.abstract

- MSigDB website (http://www.msigdb.org)
- GSEA website (https://www.gsea-msigdb.org)
